In [1]:
##Dataset url: https://grouplens.org/datasets/movielens/latest/

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.pandas.set_option('display.max_columns',None)

In [2]:
movies_df = pd.read_csv('C:/Users/DELL/Downloads/ml-latest-small/movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df=pd.read_csv('C:/Users/DELL/Downloads/ml-latest-small/ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [3]:
df=pd.merge(movies_df,rating_df,on='movieId')
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [4]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_rating=(combine_movie_rating.groupby(by=['title'])['rating'].count().reset_index()
.rename(columns={'rating':'totalRatingCount'})[['title','totalRatingCount']])

In [5]:
data=movie_rating.merge(combine_movie_rating,right_on='title',left_on='title',how='left')

In [6]:
popular_movies=data.query('totalRatingCount >= 50')

In [29]:
movie_features=popular_movies.pivot_table(index='title',columns='userId',values='rating').fillna(0)

In [8]:
#Convertion to sparse matrix
from scipy.sparse import csr_matrix
movie_features=csr_matrix(movie_features.values)

In [32]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_features)

NearestNeighbors(algorithm='brute', metric='cosine')

In [34]:
query_index = np.random.choice(movie_features.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

209


In [41]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for I, Robot (2004):

1: Day After Tomorrow, The (2004), with distance of 0.34219491481781006:
2: War of the Worlds (2005), with distance of 0.36272746324539185:
3: Matrix Revolutions, The (2003), with distance of 0.37619417905807495:
4: Matrix Reloaded, The (2003), with distance of 0.39271634817123413:
5: 300 (2007), with distance of 0.3957594633102417:
